In [23]:
import numpy as np
from datetime import datetime, timedelta
import ipaddress
import os
import json
import pandas as pd
from pandarallel import pandarallel
from joblib import Parallel, delayed
import bgpkit
import sys
import logging

pandarallel.initialize(progress_bar=True)

so = open('bgp.log', 'w', 10)
sys.stdout.echo = so
sys.stderr.echo = so

get_ipython().log.handlers[0].stream = so
get_ipython().log.setLevel(logging.INFO)


INFO: Pandarallel will run on 60 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
exploded_csv_path = '/home/mathew/incpred/csv/all_ranges_exploded.csv'
output_path = '/home/mathew/incpred/csv/features/bgp_misconfig_features.csv'

In [3]:
%%time
df = pd.read_csv(exploded_csv_path, usecols=['date', 'org', 'Ranges'])
df['Ranges'] = df['Ranges'].apply(ipaddress.IPv4Network)

CPU times: user 1min 56s, sys: 7.83 s, total: 2min 3s
Wall time: 2min 3s


In [4]:
def datetime_range(start, end, delta):
    current = start
    while current < end:
        yield current
        current += delta

start = datetime.strptime('2015-01-01', '%Y-%m-%d')
end = datetime.strptime('2022-01-02', '%Y-%m-%d')
date_list = [start + timedelta(days=x) for x in range(0, (end-start).days)]

In [ ]:
start_min = datetime.strptime('2015-01-01', '%Y-%m-%d')
end_min = datetime.strptime('2022-01-02', '%Y-%m-%d')
dts = list([dt for dt in 
       datetime_range(datetime(2015, 1, 1, 0), datetime(2022, 1, 2, 0, 14), 
       timedelta(minutes=15))])

def extract_path(elem):
    prefix = elem['prefix']
    as_path = elem['as_path']
    return f'{prefix} {as_path}'
    
def process_bgp_update_data(data):
    parser = bgpkit.Parser(url=data.url)
    announce = []
    withdraw = []
    while True:
        elem = parser.parse_next()
        if not elem:
            break
        
        elem_type = elem['elem_type']
        if elem_type == 'A':
            announce.append(extract_path(elem))
        else:
            withdraw.append(extract_path(elem))
    
    return (announce, withdraw)

def remove_old(path_dict, current_date):
    for path, date in path_dict.items():
        # only retain items that are less than 24h old
        delta = abs( (date - current_date).total_seconds() )
        if delta < 86400:
            yield (path, date)
            
def find_subnets(path, date, ranges):
    try:
        prefix = ipaddress.IPv4Network(path.split(' ')[0])
    except:
        return []
    
    result = []
    for r in ranges:
        if prefix.subnet_of(r):
            result.append( (r, date) )
    return result

broker = bgpkit.Broker()

all_announced = None
misconfig_dict = {}
if not os.path.isfile('bgp_misconfig_last_index'):
    with open('bgp_misconfig_last_index', 'w') as f:
        f.write('0')
    with open('bgp_misconfigs_cache.json', 'w') as f:
        f.write('{}')
    last_index = 0
else:
    with open('bgp_misconfig_last_index', 'r') as f:
        last_index = int(f.read())
    with open('bgp_misconfigs_cache.json', 'r') as f:
        misconfig_dict = json.load(f)
        
for i in range(len(dts)):
    if i < last_index:
        continue
        
    date = dts[i]
    d_str = date.strftime('%Y-%m-%dT%H:%M:00')
    print(d_str)
    
    date_str = date.strftime('%Y-%m-%dT%H:%M:00')
    date_plus1_str = date.strftime('%Y-%m-%dT%H:%M:01')

    items = broker.query(ts_start=date_str, ts_end=date_plus1_str, data_type='update')
    items = [x for x in items if x.collector_id.startswith('route-views')]
    res_list = Parallel(n_jobs=min(40, len(items)), verbose=1, backend='multiprocessing')(
        delayed(process_bgp_update_data)(item) 
        for item in items
    )
    (announce, withdraw) = (
        [ item for tup in res_list for item in tup[0] ],
        [ item for tup in res_list for item in tup[1] ]
    )
    announce = { path: date for path in announce }
    withdraw = { path: date for path in withdraw }
    
    print(f'number of announcements: {len(announce)}')
    
    if not all_announced:
        all_announced = announce
        continue
        
    print(f'all announce count: {len(all_announced)}')
        
    # first, remove all 24h+ paths, since these are not misconfigs according to our definition
    all_announced = dict( remove_old(all_announced, date) )
    print(f'all announce count after old removal: {len(all_announced)}')
    
    print(f'announces in common: {len(announce.keys() & all_announced.keys())}')
    
    # then, add the newly announced paths to the full list of tracked announced paths
    all_announced.update(announce)
    print(f'all announce count after adding new announce: {len(all_announced)}')
    
    # all remaining paths that are included in the withdraws are misconfigs
    # misconfig_removals = list of paths
    print(f'total withdrawn routes: {len(withdraw)}')
    misconfig_removals = list(all_announced.keys() & withdraw.keys())
    print(f'number tracked routes withdrawn: {len(misconfig_removals)}')
    
    if len(misconfig_removals) > 0:
        # filter out the ranges not relevant to us
        misconfig_removals = Parallel(n_jobs=40, verbose=1, backend='multiprocessing')(
            delayed(find_subnets)(path, all_announced[path], df[df['date'] == all_announced[path].strftime('%Y-%m-%d')]['Ranges'].tolist()) 
            for path in misconfig_removals
        )
        # misconfig_removals = dict of range->date pairs
        misconfig_removals = dict([ item for sublist in misconfig_removals for item in sublist ])
        print(f'number of relevant misconfigs: {len(misconfig_removals)}')
        print(misconfig_removals)
        
        # for each of the removed path + date, record the misconfig in the correct date
        for prefix, path_date in misconfig_removals.items():
            d = path_date.strftime('%Y-%m-%d')
            if d in misconfig_dict:
                misconfig_dict[d].append(prefix)
            else:
                misconfig_dict[d] = [prefix]
                
        with open('bgp_misconfigs_cache.json', 'w') as f:
            json.dump(misconfig_dict, f)
            
    else:
        print('no misconfigs, continuing.')
    
    
    with open('bgp_misconfig_last_index', 'w') as f:
        f.write(str(i))


2015-01-05T03:00:00


[Parallel(n_jobs=24)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    0.4s remaining:    4.0s
[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    2.2s finished


number of announcements: 331665
2015-01-05T03:15:00


[Parallel(n_jobs=24)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    0.4s remaining:    4.2s
[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    2.0s finished


number of announcements: 272740
all announce count: 331665
all announce count after old removal: 331665
announces in common: 216431
all announce count after adding new announce: 387974
total withdrawn routes: 2781
number tracked routes withdrawn: 0
no misconfigs, continuing.
2015-01-05T03:30:00


...

number of announcements: 357922
all announce count: 16220973
all announce count after old removal: 16071840
announces in common: 289482
all announce count after adding new announce: 16140280
total withdrawn routes: 5845
number tracked routes withdrawn: 0
no misconfigs, continuing.
2015-01-06T06:45:00


[Parallel(n_jobs=24)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    0.4s remaining:    4.0s
[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    3.1s finished


number of announcements: 348910
all announce count: 16140280
all announce count after old removal: 16062729
announces in common: 284709
all announce count after adding new announce: 16126930
total withdrawn routes: 5330
number tracked routes withdrawn: 0
no misconfigs, continuing.
2015-01-06T07:00:00


[Parallel(n_jobs=24)]: Using backend MultiprocessingBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   2 out of  24 | elapsed:    0.4s remaining:    4.0s
[Parallel(n_jobs=24)]: Done  24 out of  24 | elapsed:    2.7s finished


number of announcements: 307156
all announce count: 16126930


In [22]:
misconfig_dict

,date,0,1
0,2015-01-01,77.61.161.80/29,77.61.161.0/24


In [ ]:
misconfigs_dict

In [ ]:
misconfig_df = (pd.DataFrame.from_dict(misconfigs_dict, orient='index')
                .reset_index(names='date')
                .rename(columns={0: 'bgp_path_misconfigs'}))
misconfig_df

In [ ]:
misconfig_df.to_csv('csv/features/bgp_misconfig_features.csv', index=False)